# Fewshot Token Classification using spacy-llm

This tutorial will guide you through a Named Entity Recognition (NER) pipeline using spacy-llm and fewshot learning. The steps will be as follows:

- Run Argilla and load `spacy-llm` along with other libraries
- Define fewshot examples for the model
- Define config for your pipeline and initiliaze it
- Create `TokenClassificationRecord` and store the inferences in it
- Push to Argilla

## Introduction

NER is a Natural Language Processing task where tokens are categorized into pre-defined categories such as person, organization or place as `named entities`. Thus, NER is one of the primary steps to turn the unstructured data into structured data. With Argilla, you will have the opportunity to validate and annotate the named entities that you obtain from your NER pipeline.

[spacy-llm](https://spacy.io/usage/large-language-models) is a package that integrates the strength of LLMs into regular spaCy pipelines, thus allowing to quick and practical prompting for various tasks. Besides, since it requires no training data, the models are ready to use in your pipeline. If you want to train your own model or create your custom task, `spacy-llm` also helps to implement any custom pipeline.\n",

## Running Argilla

For this tutorial, you will need to have an Argilla server running. There are two main options for deploying and running Argilla:


**Deploy Argilla on Hugging Face Spaces**: If you want to run tutorials with external notebooks (e.g., Google Colab) and you have an account on Hugging Face, you can deploy Argilla on Spaces with a few clicks:

[![deploy on spaces](https://huggingface.co/datasets/huggingface/badges/raw/main/deploy-to-spaces-lg.svg)](https://huggingface.co/new-space?template=argilla/argilla-template-space)

For details about configuring your deployment, check the [official Hugging Face Hub guide](https://huggingface.co/docs/hub/spaces-sdks-docker-argilla).


**Launch Argilla using Argilla's quickstart Docker image**: This is the recommended option if you want [Argilla running on your local machine](../../getting_started/quickstart.ipynb). Note that this option will only let you run the tutorial locally and not with an external notebook service.

For more information on deployment options, please check the Deployment section of the documentation.

<div class="alert alert-info">

Tip
    
This tutorial is a Jupyter Notebook. There are two options to run it:

- Use the Open in Colab button at the top of this page. This option allows you to run the notebook directly on Google Colab. Don't forget to change the runtime type to GPU for faster model training and inference.
- Download the .ipynb file by clicking on the View source link at the top of the page. This option allows you to download the notebook and run it on your local machine or on a Jupyter notebook tool of your choice.
</div>

## Setup

You will need to install some libraries along with Argilla for this tutorial:

In [ ]:
!pip install "spacy-llm[transformers]" argilla datasets -qqq